In [1]:
from bertopic import BERTopic
from sklearn.feature_extraction.text import CountVectorizer
import json
import pandas as pd
import os
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
import string

C:\Users\simon\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\umap\distances.py:1063: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  @numba.jit()
C:\Users\simon\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\umap\distances.py:1071: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecatio

In [2]:
os. chdir("c:\\Users\\simon\\OneDrive\\Dokumente\\UNILU\\3 - HS24\\1 MA\\business-reports-nlp")

def read_text_files_in_folder(folder_path):
    data = []
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.endswith(".txt"):
                full_path = os.path.join(root, file)
                with open(full_path, 'r', encoding='utf-8') as f:
                    content = f.read()
                data.append({
                    'file_name': file,
                    'file_path': full_path,
                    'content': content
                })
                
    return pd.DataFrame(data)

# Example usage
folder_path = "reports_digipay_txt/consultancies_pdfminer_cleaned"#"examples"
df = read_text_files_in_folder(folder_path)
texts = df['content'].to_list()

In [3]:
def clean_text(text):
    # Replace multiple spaces with a single space
    text = re.sub(' +', ' ', text)
    
    # Ensure there is exactly one space after each punctuation mark
    text = re.sub(r'(?<=[!?.;,:])(?=[^\s])', ' ', text)
    
    # Convert text to lowercase
    text = text.lower()
    
    # Tokenize the text
    tokens = word_tokenize(text)
    
    # Remove stopwords
    #stop_words = set(stopwords.words('english'))
    #tokens = [word for word in tokens if word not in stop_words]
    
    # Stem the words
    ps = PorterStemmer()
    tokens = [ps.stem(word) for word in tokens]
    
    # Remove punctuation and numbers
    tokens = [word for word in tokens if word.isalpha()]
    
    # Convert tokens back to string
    cleaned_text = ' '.join(tokens)
    
    return cleaned_text


# Apply text cleaning to 'content' column
texts = df['content'].apply(clean_text).to_list()

# Initialize an empty list to hold the individual sentences
sentences = []

# Iterate over each long text in the original list
for text in texts:
    # Use regular expression to split text into sentences
    splitters = re.split(r'(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s', text)
    # Extend the new list with the individual sentences
    sentences.extend(splitters)

In [4]:
# this is to remove stopwords, for lower columes of data stopwords can cause issues.
vectorizer_model = CountVectorizer(ngram_range=(1, 2), stop_words="english")

topic_model = BERTopic(
    vectorizer_model=vectorizer_model,
    language='english',
    calculate_probabilities=True,
    verbose=True
)
topics, probs = topic_model.fit_transform(sentences)
#topic_model = BERTopic.load("arxiv_bertopic")

Batches:   0%|          | 0/4 [00:00<?, ?it/s]

2023-09-20 18:43:26,311 - BERTopic - Transformed documents to Embeddings
Exception ignored on calling ctypes callback function: <function ExecutionEngine._raw_object_cache_notify at 0x00000259CA282170>
Traceback (most recent call last):
  File "C:\Users\simon\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\llvmlite\binding\executionengine.py", line 171, in _raw_object_cache_notify
    def _raw_object_cache_notify(self, data):
KeyboardInterrupt: 
2023-09-20 18:43:34,528 - BERTopic - Reduced dimensionality
2023-09-20 18:43:34,553 - BERTopic - Clustered reduced embeddings


In [5]:
topic_model.get_topic_info()#.head(10)

,Topic,Count,Name,Representation,Representative_Docs
0,-1,39,-1_payment_digit_global_percent,"[payment, digit, global, percent, mckinsey, se...",[digit financ for all power inclus growth in e...
1,0,49,0_payment_bank_custom_servic,"[payment, bank, custom, servic, transact, mark...",[tabl of content prefac transact analysi key f...
2,1,21,1_payment_bank_revenu_percent,"[payment, bank, revenu, percent, growth, globa...",[global payment strong fundament despit uncert...
3,2,15,2_mobil_payment_consum_mobil payment,"[mobil, payment, consum, mobil payment, provid...",[card a look at the near field commun payment ...


In [12]:
for i in range(5):
    print(f"{topics[i]}: {df['content'][i]}")

2: Consumer Mobile Payments Survey

Driving Value and Adoption  of Mobile Payments—  Consumers Want More

Accenture recently surveyed North American  smart phone users to understand how consumers  can be encouraged to make mobile payments.  After all, persuading more consumers to use  their smart phones to pay for goods and services,   whether they are buying a tank of gas, a  morning cup of coffee or a night out at the  movies, presents a huge opportunity. 

A recent report by IDC Financial Insights  predicted that the global value of proximity  payments, both Near Field Communication  (NFC or tap-and-pay) and mobile bar code  transactions, will reach $296 billion by 20171.

Financial institutions, merchants, payment  processors and technology providers have  invested heavily in online channels in recent  years, and as a result, have seen a rapid  growth in online payments, enhanced  customer loyalty and increased enterprise  profitability. Harnessing the constant,  real-time interac

In [28]:
# Generate Labels
topic_labels = topic_model.generate_topic_labels(nr_words=3, topic_prefix=False, word_length=15, separator=" - ")
topic_model.set_topic_labels(topic_labels)

# Create own Labels
#topic_model.set_topic_labels({0: "Reinforcement Learning", 8: "Transformer models"})
topic_model.get_topic_info().head(10)

,Topic,Count,Name,CustomName,Representation,Representative_Docs
0,-1,9,-1_global_percent_skill_digit,global - percent - skill,"[global, percent, skill, digit, flow, job, mck...",[h e f u u r e f w r k n e r c p e p l e n p l...
1,0,101,0_payment_bank_custom_servic,payment - bank - custom,"[payment, bank, custom, servic, market, transa...",[world payment report tabl content prefac sect...
2,1,14,1_mobil_payment_servic_bank,mobil - payment - servic,"[mobil, payment, servic, bank, custom, market,...",[road mobil payment servic road mobil payment ...


In [ ]:
# Change n-grams without re-fitting our entire model
topic_model.update_topics(texts, n_gran_range=(1,3))

# Merge specific topics
topic_model.merge_topics(texts, topics_to_merge=[1, 8, 12])

# Reduce the number of topics by iteratively merging them
topic_model.reduce_topics(texts, nr_topics=100)

# Find specific topics
topic_model.find_topics("digital payment", top_n=1)

In [8]:
topic_model.visualize_barchart(width=200, height=330, top_n_topics=4, n_words=10)

In [10]:
topic_model.visualize_heatmap(n_clusters=2)

In [15]:
topic_model.visualize_topics()

ValueError: zero-size array to reduction operation maximum which has no identity